In [1]:
# @inproceedings{brachmann2019ngransac,
#   title={{N}eural- {G}uided {RANSAC}: {L}earning Where to Sample Model Hypotheses},
#   author={Brachmann, Eric and Rother, Carsten},
#   booktitle={ICCV},
#   year={2019}
# }

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/ngdsac_horizon/
from google.colab import files

from hlw_dataset import HLWDataset
from model import Model
import torchvision.utils as vutils

from skimage.io import imsave
import skimage.io as io
from skimage import color
from skimage.io import imsave
from skimage.draw import line, set_color, circle

import time
import warnings
import argparse
import os

from ngdsac import NGDSAC
from loss import Loss

import cv2

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/ngdsac_horizon


In [4]:
# parser = argparse.ArgumentParser(description='Train a horizon line estimation network on the HLW dataset using (NG-)DSAC.', formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# parser.add_argument('--session', '-sid', default='', 
# 	help='custom session name appended to output files, useful to separate different runs of the program')

# parser.add_argument('--capacity', '-c', type=int, default=4, 
# 	help='controls the model capactiy of the network by scaling the number of channels in each layer')

# parser.add_argument('--imagesize', '-is', type=int, default=256, 
# 	help='rescale images to this max side length')

# parser.add_argument('--inlierthreshold', '-it', type=float, default=0.05, 
# 	help='threshold used in the soft inlier count, relative to image size (1 = image width)')

# parser.add_argument('--inlieralpha', '-ia', type=float, default=0.1, 
# 	help='scaling factor for the soft inlier scores (controls the peakiness of the hypothesis distribution)')

# parser.add_argument('--inlierbeta', '-ib', type=float, default=100.0, 
# 	help='scaling factor within the sigmoid of the soft inlier count')

# parser.add_argument('--storeinterval', '-si', type=int, default=1000, 
# 	help='store network weights and a prediction vizualisation every x training iterations')

# parser.add_argument('--hypotheses', '-hyps', type=int, default=16, 
# 	help='number of line hypotheses sampled for each image')

# parser.add_argument('--batchsize', '-bs', type=int, default=32, 
# 	help='training batch size')

# parser.add_argument('--learningrate', '-lr', type=float, default=0.0001, 
# 	help='learning rate')

# parser.add_argument('--iterations', '-i', type=int, default=250000, 
# 	help='number of training iterations (parameter updates)')

# parser.add_argument('--scheduleoffset', '-soff', type=int, default=150000, 
# 	help='start learning rate schedule ofter this many iterations')

# parser.add_argument('--schedulestep', '-sstep', type=int, default=25000, 
# 	help='half learning rate ofter this many iterations')

# parser.add_argument('--samplesize', '-ss', type=int, default=2, 
# 	help='number of ng-dsac runs for each training image to approximate the expectation when learning neural guidance')

# parser.add_argument('--invalidloss', '-il', type=float, default=1, 
# 	help='penalty for sampling invalid hypotheses')

# parser.add_argument('--uniform', '-u', action='store_true', 
# 	help='disable neural-guidance and sample data points uniformely; corresponds to a DSAC model')

#opt = parser.parse_args()

session = 'colab_baseline_26'
capacity = 4
imagesize = 256
inlierthreshold = 0.05
inlieralpha = 0.1
inlierbeta = 100.0
storeinterval = 1000
hypotheses = 16
batchsize = 32
learningrate = 0.0002
iterations = 10000				# Default = 250000
scheduleoffset = 150000
schedulestep = 25000
samplesize = 2
invalidloss = 1
uniform = False

#Directories
tngDir = '/content/drive/My Drive/Colab Notebooks/ngdsac_horizon/hlw_1_2/split/'


# setup training set
#trainset = HLWDataset('hlw/split/train.txt', imagesize, training=True)

splitFileName = 'train.txt'
path = tngDir + splitFileName
trainset = HLWDataset(path, imagesize, training=True)
trainset_loader = torch.utils.data.DataLoader(trainset, shuffle=True, num_workers=12, batch_size=batchsize)  ##Set workers to 0 from 6 (multithreading issue)

# setup ng dsac estimator
loss = Loss(imagesize) 
ngdsac = NGDSAC(hypotheses, inlierthreshold, inlierbeta, inlieralpha, loss, invalidloss)

# setup network
nn = Model(capacity)
nn.train()
nn = nn.cuda()
#plot_model(nn, to_file="/content/Model/model.png", show_shapes=True, show_layer_names=True)

# optimizer and lr schedule (schedule offset handled further below)
optimizer = optim.Adam(nn.parameters(), lr=learningrate)

# Removing automatic optimizer in pursuit of scholarly excellence
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=schedulestep, gamma=0.5)

# keep track of training progress
train_log = open('log_'+session+'.txt', 'w')
	#uploaded = drive.CreateFile({'title': 'Sample file.txt'})
train_log.write('Sample upload file content')
#files.download('log_'+session+'.txt')
print('Downloaded ' + 'log_'+session+'.txt')

iteration = 0
epoch = 0

while True:

	print('=== Epoch: ', epoch, '========================================')

	for inputs, labels, xStart, xEnd, imh, idx in trainset_loader:

		start_time = time.time()

		# predict points and neural guidance
		inputs = inputs.cuda()
		points, log_probs = nn(inputs)

		if uniform:
			# overwrite neural guidance with uniform sampling probabilities
			log_probs.fill_(1/log_probs.size(1))
			log_probs = torch.log(log_probs)

		g_log_probs = torch.zeros(log_probs.size()).cuda() # gradients for neural guidance
		g_points = torch.zeros(points.size()).cuda() # gradients for point positions

		# approximate neural guidance expectation by sampling

		exp_loss = 0 # mean loss over samples
		losses = [] # losses per sample, we will substract the mean loss later as baseline
		sample_grads = [] # gradients per sample

		for s in range(samplesize):

			# fit lines with ngdsac (also calculates expected loss for DSAC)
			cur_loss = ngdsac(points, log_probs, labels, xStart, xEnd, imh)

			# calculate gradients for 2D point predictions by PyTorch (autograd of expected loss)
			g_points += torch.autograd.grad(cur_loss, points)[0]
			# gradients for neural guidance have been calculated by NGDSAC
			sample_grads.append(ngdsac.g_log_probs.cuda() / batchsize)

			exp_loss += cur_loss
			losses.append(cur_loss)

		g_points /= samplesize
		exp_loss /= samplesize

		# subtract baseline (mean over samples) for neural guidance gradients to reduce variance
		for i,l in enumerate(losses):
			g_log_probs += sample_grads[i] * (float(l) - float(exp_loss))
		g_log_probs /= samplesize * 10
		g_log_probs = g_log_probs.cuda()

		if uniform:
			# DSAC, no neural guidance
			torch.autograd.backward((points), (g_points))
		else:
			# full NG-DSAC
			torch.autograd.backward((points, log_probs), (g_points, g_log_probs))
		
		# # Removing automatic optimizer in pursuit of scholarly excellence
		# optimizer.step() 
		# # apply learning rate schedule
		# if iteration >= scheduleoffset:
		# 	scheduler.step()
		# optimizer.zero_grad()

		optimizer.step()
		optimizer.zero_grad() 

		# wrap up
		end_time = time.time()-start_time
		print('Iteration: %6d, Exp. Loss: %2.2f, Time: %.2fs' % (iteration, exp_loss, end_time), flush=True)

		#train_log.write('%d %f\n' % (iteration, exp_loss))

		if iteration % storeinterval == 0:
			torch.save(nn.state_dict(), './weights_' + session + '.net')
			#files.download('./weights_' + session + '.net')

		del exp_loss, points, log_probs, g_log_probs, g_points, losses, sample_grads

		iteration += 1

		if iteration > iterations:
			break

	epoch += 1

	if iteration > iterations:
		break

print('Done without errors.')
train_log.close()
files.download('log_'+session+'.txt')
files.download('./weights_' + session + '.net')


Downloaded log_colab_baseline_26.txt
=== Epoch:  0 ========================================
Iteration:      0, Exp. Loss: 0.53, Time: 2.64s
Iteration:      1, Exp. Loss: 0.62, Time: 2.92s
Iteration:      2, Exp. Loss: 0.66, Time: 3.19s
Iteration:      3, Exp. Loss: 0.62, Time: 3.49s
Iteration:      4, Exp. Loss: 0.45, Time: 3.09s
Iteration:      5, Exp. Loss: 0.46, Time: 2.91s
Iteration:      6, Exp. Loss: 0.42, Time: 3.39s
Iteration:      7, Exp. Loss: 0.43, Time: 3.04s
Iteration:      8, Exp. Loss: 0.40, Time: 3.21s
Iteration:      9, Exp. Loss: 0.37, Time: 3.51s
Iteration:     10, Exp. Loss: 0.38, Time: 3.06s
Iteration:     11, Exp. Loss: 0.37, Time: 3.17s
Iteration:     12, Exp. Loss: 0.36, Time: 3.57s
Iteration:     13, Exp. Loss: 0.36, Time: 3.41s
Iteration:     14, Exp. Loss: 0.32, Time: 3.07s
Iteration:     15, Exp. Loss: 0.30, Time: 2.88s
Iteration:     16, Exp. Loss: 0.35, Time: 3.32s
Iteration:     17, Exp. Loss: 0.31, Time: 3.16s
Iteration:     18, Exp. Loss: 0.31, Time: 3.

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8460 bytes but only got 4409. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag)


Iteration:    176, Exp. Loss: 0.28, Time: 3.33s
Iteration:    177, Exp. Loss: 0.23, Time: 3.29s
Iteration:    178, Exp. Loss: 0.22, Time: 2.96s
Iteration:    179, Exp. Loss: 0.26, Time: 3.24s
Iteration:    180, Exp. Loss: 0.31, Time: 2.91s
Iteration:    181, Exp. Loss: 0.26, Time: 3.03s
Iteration:    182, Exp. Loss: 0.25, Time: 3.10s
Iteration:    183, Exp. Loss: 0.24, Time: 3.09s
Iteration:    184, Exp. Loss: 0.28, Time: 2.91s
Iteration:    185, Exp. Loss: 0.22, Time: 3.21s
Iteration:    186, Exp. Loss: 0.21, Time: 3.21s
Iteration:    187, Exp. Loss: 0.29, Time: 2.90s
Iteration:    188, Exp. Loss: 0.24, Time: 3.28s
Iteration:    189, Exp. Loss: 0.24, Time: 3.16s
Iteration:    190, Exp. Loss: 0.31, Time: 3.31s
Iteration:    191, Exp. Loss: 0.22, Time: 3.50s
Iteration:    192, Exp. Loss: 0.20, Time: 3.14s
Iteration:    193, Exp. Loss: 0.23, Time: 3.65s
Iteration:    194, Exp. Loss: 0.27, Time: 3.11s
Iteration:    195, Exp. Loss: 0.24, Time: 3.45s
Iteration:    196, Exp. Loss: 0.26, Time

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41730
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag)


Iteration:    419, Exp. Loss: 0.29, Time: 3.49s
Iteration:    420, Exp. Loss: 0.23, Time: 2.77s
Iteration:    421, Exp. Loss: 0.23, Time: 3.05s
Iteration:    422, Exp. Loss: 0.23, Time: 3.20s
Iteration:    423, Exp. Loss: 0.22, Time: 3.33s
Iteration:    424, Exp. Loss: 0.27, Time: 3.21s
Iteration:    425, Exp. Loss: 0.20, Time: 2.88s
Iteration:    426, Exp. Loss: 0.21, Time: 2.66s
Iteration:    427, Exp. Loss: 0.20, Time: 3.28s
Iteration:    428, Exp. Loss: 0.22, Time: 3.05s
Iteration:    429, Exp. Loss: 0.21, Time: 3.07s
Iteration:    430, Exp. Loss: 0.24, Time: 3.25s
Iteration:    431, Exp. Loss: 0.21, Time: 2.95s
Iteration:    432, Exp. Loss: 0.23, Time: 3.20s
Iteration:    433, Exp. Loss: 0.23, Time: 3.27s
Iteration:    434, Exp. Loss: 0.20, Time: 3.48s
Iteration:    435, Exp. Loss: 0.20, Time: 2.91s
Iteration:    436, Exp. Loss: 0.22, Time: 3.60s
Iteration:    437, Exp. Loss: 0.25, Time: 2.99s
Iteration:    438, Exp. Loss: 0.20, Time: 3.34s
Iteration:    439, Exp. Loss: 0.22, Time

...

Iteration:   9874, Exp. Loss: 0.07, Time: 3.14s
Iteration:   9875, Exp. Loss: 0.13, Time: 3.60s
Iteration:   9876, Exp. Loss: 0.10, Time: 2.99s
Iteration:   9877, Exp. Loss: 0.12, Time: 3.29s
Iteration:   9878, Exp. Loss: 0.12, Time: 3.80s
Iteration:   9879, Exp. Loss: 0.12, Time: 3.29s
Iteration:   9880, Exp. Loss: 0.07, Time: 3.13s
Iteration:   9881, Exp. Loss: 0.12, Time: 3.47s
Iteration:   9882, Exp. Loss: 0.09, Time: 3.12s
Iteration:   9883, Exp. Loss: 0.11, Time: 3.63s
Iteration:   9884, Exp. Loss: 0.15, Time: 3.49s
Iteration:   9885, Exp. Loss: 0.09, Time: 3.53s
Iteration:   9886, Exp. Loss: 0.11, Time: 3.09s
Iteration:   9887, Exp. Loss: 0.10, Time: 3.98s
Iteration:   9888, Exp. Loss: 0.10, Time: 3.35s
Iteration:   9889, Exp. Loss: 0.10, Time: 3.35s
Iteration:   9890, Exp. Loss: 0.07, Time: 3.73s
Iteration:   9891, Exp. Loss: 0.08, Time: 3.24s
Iteration:   9892, Exp. Loss: 0.10, Time: 3.61s
Iteration:   9893, Exp. Loss: 0.14, Time: 3.35s
Iteration:   9894, Exp. Loss: 0.12, Time

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# @inproceedings{brachmann2019ngransac,
#   title={{N}eural- {G}uided {RANSAC}: {L}earning Where to Sample Model Hypotheses},
#   author={Brachmann, Eric and Rother, Carsten},
#   booktitle={ICCV},
#   year={2019}
# }

import torch
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/ngdsac_horizon/

from hlw_dataset import HLWDataset
from model import Model

import time
import argparse

from ngdsac import NGDSAC
from loss import Loss

# parser = argparse.ArgumentParser(description='Test a trained horizon line estimation network (DSAC or NG-DSAC).', formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# parser.add_argument('model', type=str,
# 	help='a trained network')

# parser.add_argument('--capacity', '-c', type=int, default=4, 
# 	help='controls the model capactiy of the network, must match the model to load (multiplicative factor for number of channels)')

# parser.add_argument('--imagesize', '-is', type=int, default=256, 
# 	help='size of input images to the network, must match the model to load')

# parser.add_argument('--inlierthreshold', '-it', type=float, default=0.05, 
# 	help='threshold used in the soft inlier count, relative to image size')

# parser.add_argument('--inlieralpha', '-ia', type=float, default=0.1, 
# 	help='scaling factor for the soft inlier scores (controls the peakiness of the hypothesis distribution)')

# parser.add_argument('--inlierbeta', '-ib', type=float, default=100.0, 
# 	help='scaling factor within the sigmoid of the soft inlier count')

# parser.add_argument('--hypotheses', '-hyps', type=int, default=16, 
# 	help='number of line hypotheses sampled for each image')

# parser.add_argument('--session', '-sid', default='', 
# 	help='custom session name appended to output files; useful to separate different runs of the program')

# parser.add_argument('--invalidloss', '-il', type=int, default=1, 
# 	help='penalty for sampling invalid hypotheses')

# parser.add_argument('--uniform', '-u', action='store_true', 
# 	help='disable neural-guidance and sample data points uniformely; corresponds to a DSAC model')

#Arguments

model = '/content/drive/My Drive/Colab Notebooks/ngdsac_horizon/models/weights_colab_baseline_26.net'
capacity = 4
imagesize = 256
inlierthreshold = 0.05
inlieralpha = 0.1
inlierbeta = 100.0
hypotheses = 16
session = 'test26'
invalidloss = 1
uniform = False


#opt = parser.parse_args()

testDir = '/content/drive/My Drive/Colab Notebooks/ngdsac_horizon/hlw_1_2/split/'

# setup test set
testset = HLWDataset(testDir + 'test.txt', imagesize, training=False)
testset_loader = torch.utils.data.DataLoader(testset, shuffle=False, num_workers=6, batch_size=1)

# setup ng dsac estimator
loss = Loss(imagesize, cut_off = 100) 
ngdsac = NGDSAC(hypotheses, inlierthreshold, inlierbeta,inlieralpha, loss,invalidloss)

# load network
nn = Model(capacity)
nn.load_state_dict(torch.load(model))
nn.eval()
nn = nn.cuda()

# write test results
test_log = open('test_'+session+'.txt', 'w', 1)

def AUC(losses, thresholds, binsize):
	"""Compute the AUC up to a set of error thresholds.
	Return mutliple AUC corresponding to multiple threshold provided.
	Keyword arguments:
	losses -- list of losses which the AUC should be calculated for
	thresholds -- list of threshold values up to which the AUC should be calculated
	binsize -- bin size to be used fo the cumulative histogram when calculating the AUC, the finer the more accurate
	"""

	bin_num = int(max(thresholds) / binsize)
	bins = np.arange(bin_num + 1) * binsize  

	hist, _ = np.histogram(losses, bins) # histogram up to the max threshold
	hist = hist.astype(np.float32) / len(losses) # normalized histogram
	hist = np.cumsum(hist) # cumulative normalized histogram
	 
	# calculate AUC for each threshold
	return [np.mean(hist[:int(t / binsize)]) for t in thresholds]

losses = []

for inputs, labels, xStart, xEnd, imh, idx in testset_loader:

	start_time = time.time()

	with torch.no_grad():
		# forward pass of neural network
		points, log_probs = nn(inputs.cuda())

		if uniform:
			# overwrite neural guidance with uniform sampling probabilities
			log_probs.fill_(1/log_probs.size(1))
			log_probs = torch.log(log_probs)

		# fit line with NG-DSAC
		ngdsac(points, log_probs, labels, xStart, xEnd, imh) 

	# evaluate (assumes a batch size of 1)
	cur_loss = loss(ngdsac.est_parameters[0], labels[0], xStart[0], xEnd[0], imh[0])

	# wrap up
	end_time = time.time()-start_time
	print('Image: %s, Loss: %2.2f, Time: %.2fs' 
		% (testset.images[idx[0]], cur_loss, end_time), flush=True)

	test_log.write('%s %f\n' % (testset.images[idx[0]], cur_loss))
	losses.append(cur_loss)

auc = AUC(losses, thresholds=[0.25], binsize=0.0001)

print("\n==========================================")
print("AUC@0.25: %.1f%%" % (auc[0]*100))
print("==========================================\n")
print('Done without errors.')

test_log.close()
files.download('test_'+session+'.txt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/ngdsac_horizon
Image: /content/drive/My Drive/Colab Notebooks/ngdsac_horizon/hlw_1_2/images/Alamo/10047400_4909449851_o.jpg, Loss: 0.10, Time: 0.05s
Image: /content/drive/My Drive/Colab Notebooks/ngdsac_horizon/hlw_1_2/images/Alamo/110935763_66fc01689c_o.jpg, Loss: 0.21, Time: 0.02s
Image: /content/drive/My Drive/Colab Notebooks/ngdsac_horizon/hlw_1_2/images/Alamo/110935776_5625ee33a9_o.jpg, Loss: 0.09, Time: 0.03s
Image: /content/drive/My Drive/Colab Notebooks/ngdsac_horizon/hlw_1_2/images/Alamo/1157929690_376fa9d177_o.jpg, Loss: 0.09, Time: 0.02s
Image: /content/drive/My Drive/Colab Notebooks/ngdsac_horizon/hlw_1_2/images/Alamo/1157940322_c3423c93a7_o.jpg, Loss: 0.08, Time: 0.02s
Image: /content/drive/My Drive/Colab Notebooks/ngdsac_horizon/hlw_1_2/images/Alamo/1190606546_daf2749711_o.jpg, Loss: 0.05, Time: 0.02s
Imag

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Demo

In [19]:
# @inproceedings{brachmann2019ngransac,
#   title={{N}eural- {G}uided {RANSAC}: {L}earning Where to Sample Model Hypotheses},
#   author={Brachmann, Eric and Rother, Carsten},
#   booktitle={ICCV},
#   year={2019}
# }


model = '/content/drive/My Drive/Colab Notebooks/ngdsac_horizon/models/weights_colab_baseline_2.net'
capacity = 4
imagesize = 256
inlierthreshold = 0.05
inlieralpha = 0.1
inlierbeta = 100.0
hypotheses = 16
invalidloss = 1
uniform = False
scorethreshold = 0.4
verbose = True

input = '/content/drive/My Drive/Colab Notebooks/ngdsac_horizon/test/2.jpg'
outputName = '/2.png'


output_folder = '/content/drive/My Drive/Colab Notebooks/ngdsac_horizon/test/results'
#if not os.path.isdir(output_folder): os.makedirs(output_folder)

# setup ng dsac estimator
ngdsac = NGDSAC(hypotheses, inlierthreshold, inlierbeta, inlieralpha, Loss(imagesize), 1)

# load network
nn = Model(capacity)
nn.load_state_dict(torch.load(model))
nn.eval()
nn = nn.cuda()


def process_frame(image):
	'''
	Estimate horizon line for an image and return a visualization.

	image -- 3 dim numpy image tensor
	'''

	# determine image scaling factor
	image_scale = max(image.shape[0], image.shape[1])
	image_scale = imagesize / image_scale 

	# convert image to RGB
	if len(image.shape) < 3:
		image = color.gray2rgb(image)

	# store original image dimensions		
	src_h = int(image.shape[0] * image_scale)
	src_w = int(image.shape[1] * image_scale)

	# resize and to gray scale
	image = transforms.functional.to_pil_image(image)
	image = transforms.functional.resize(image, (src_h, src_w))
	image = transforms.functional.adjust_saturation(image, 0)
	image = transforms.functional.to_tensor(image)

	# make image square by zero padding
	padding_left = int((imagesize - image.size(2)) / 2)
	padding_right = imagesize - image.size(2) - padding_left
	padding_top = int((imagesize - image.size(1)) / 2)
	padding_bottom = imagesize - image.size(1) - padding_top

	padding = torch.nn.ZeroPad2d((padding_left, padding_right, padding_top, padding_bottom))
	image = padding(image)

	image_src = image.clone().unsqueeze(0)

	# normalize image (mean and variance), values estimated offline from HLW training set
	img_mask = image.sum(0) > 0
	image[:,img_mask] -= 0.45
	image[:,img_mask] /= 0.25
	image = image.unsqueeze(0).cuda()

	with torch.no_grad():
		#predict data points and neural guidance
		points, log_probs = nn(image)
	
		if uniform:
			# overwrite neural guidance with uniform sampling probabilities
			log_probs.fill_(1/log_probs.size(1))
			log_probs = torch.log(log_probs)

		# fit line with NG-DSAC, providing dummy ground truth labels
		ngdsac(points, log_probs, torch.zeros((1,2)), torch.zeros((1)), torch.ones((1)), torch.ones((1))) 

	def draw_line(data, lX1, lY1, lX2, lY2, clr):
		'''
		Draw a line with the given color and opacity.

		data -- image to draw to
		lX1 -- x value of line segment start point
		lY1 -- y value of line segment start point
		lX2 -- x value of line segment end point
		lY2 -- y value of line segment end point
		clr -- line color, triple of values
		'''

		rr, cc = line(lY1, lX1, lY2, lX2)
		set_color(data, (rr, cc), clr)

	def draw_models(labels, clr, data):
		'''
		Draw circles for a batch of images.
	
		labels -- line parameters, array shape (Nx2) where 
			N is the number of images in the batch
			2 is the number of line parameters (offset,  slope)
		data -- batch of images to draw to
		'''

		# number of image in batch
		n = labels.shape[0]

		for i in range (n):

			#line
			lY1 = int(labels[i, 0] * imagesize)
			lY2 = int(labels[i, 1] * imagesize + labels[i, 0] * imagesize)
			draw_line(data[i], 0, lY1, imagesize, lY2, clr)

		return data	

	def draw_wpoints(points, data, weights, clrmap):
		'''
		Draw 2D points for a batch of images.

		points -- 2D points, array shape (Nx2xM) where 
			N is the number of images in the batch
			2 is the number of point dimensions (x, y)
			M is the number of points
		data -- batch of images to draw to
		weights -- array shape (NxM), one weight per point, for visualization
		clrmap -- OpenCV color map for visualizing weights
			
		'''

		# create explicit color map
		color_map = np.arange(256).astype('u1')
		color_map = cv2.applyColorMap(color_map, clrmap)
		color_map = color_map[:,:,::-1] # BGR to RGB

		n = points.shape[0] # number of images
		m = points.shape[2] # number of points

		for i in range (0, n):

			s_idx = weights[i].sort(descending=False)[1] # draw low weight points first
			weights[i] = weights[i] / weights[i].max() # normalize weights for visualization

			for j in range(0, m):

				idx = int(s_idx[j])

				# convert weight to color
				clr_idx = float(min(1, weights[i,idx]))
				clr_idx = int(clr_idx * 255)
				clr = color_map[clr_idx, 0] / 255

				# draw point
				r = int(points[i, 0, idx] * imagesize)
				c = int(points[i, 1, idx] * imagesize)
				rr, cc = circle(r, c, 2)
				set_color(data[i], (rr, cc), clr)

		return data

	# normalized inlier score of the estimated line
	score = ngdsac.batch_inliers[0].sum() / points.shape[2]

	image_src = image_src.cpu().permute(0,2,3,1).numpy() #Torch to Numpy
	viz_probs = image_src.copy() * 0.2 # make a faint copy of the input image
	
	# draw estimated line
	if score > scorethreshold:
		image_src = draw_models(ngdsac.est_parameters, clr=(0,0,1), data=image_src)

	viz = [image_src]

	if verbose:	
		# create additional visualizations

		# draw faint estimated line 
		viz_score = viz_probs.copy()
		viz_probs = draw_models(ngdsac.est_parameters, clr=(0.3,0.3,0.3), data=viz_probs)
		viz_inliers = viz_probs.copy()

		# draw predicted points with neural guidance and soft inlier count, respectively
		viz_probs = draw_wpoints(points, viz_probs, weights=torch.exp(log_probs), clrmap=cv2.COLORMAP_PLASMA)
		viz_inliers = draw_wpoints(points, viz_inliers, weights=ngdsac.batch_inliers, clrmap=cv2.COLORMAP_WINTER)

		# create a explicit color map for visualize score of estimate line
		color_map = np.arange(256).astype('u1')
		color_map = cv2.applyColorMap(color_map, cv2.COLORMAP_HSV)	
		color_map = color_map[:,:,::-1]

		# map score to color
		score = int(score*100) #using only the first portion of HSV to get a nice (red, yellow, green) gradient
		clr = color_map[score, 0] / 255

		viz_score = draw_models(ngdsac.est_parameters, clr=clr, data=viz_score)

		viz = viz + [viz_probs, viz_inliers, viz_score]

	#undo zero padding of inputs
	if padding_left > 0:
		viz = [img[:,:,padding_left:,:] for img in viz]
	if padding_right > 0:
		viz = [img[:,:,:-padding_right,:] for img in viz]
	if padding_top > 0:
		viz = [img[:,padding_top:,:,:] for img in viz]
	if padding_bottom > 0:
		viz = [img[:,:-padding_bottom,:,:] for img in viz]		

	# convert to a single uchar image
	viz = np.concatenate(viz, axis=2)
	viz = viz * 255
	viz = viz.astype('u1')

	return viz[0]

# try to read input as image
image = cv2.imread(input)

if image is not None:
	#success, it was an image
	viz = process_frame(image)
	imsave(output_folder + outputName, viz)

else:
	#failure, try interpreting it as video
	cap = cv2.VideoCapture(input)
	iteration = 0

	while(cap.isOpened()):
		ret, image = cap.read()

		if not ret:
			break

		print("Processing frame %5d." % iteration)
	
		viz = process_frame(image)
		imsave(output_folder + '/frame_' + str(iteration).zfill(5) + '.png', viz)

		iteration = iteration + 1
